In [1]:
!pip install optuna
import pandas as pd
import numpy as np
import sklearn
import optuna  # pip install optuna


# def objective(trial):
#     x = trial.suggest_float("x", -7, 7)
#     y = trial.suggest_float("y", -7, 7)
#     return (x - 1) ** 2 + (y + 3) ** 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 5.4 MB/s eta 0:00:00


In [2]:
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_validate, train_test_split

# Load data
penguins = sns.load_dataset("penguins").dropna()
X, y = penguins.drop("body_mass_g", axis=1), penguins[["body_mass_g"]]

# OH encode categoricals
X = pd.get_dummies(X)

# Init model with defaults
gr_reg = GradientBoostingRegressor()

# kf = KFold(n_splits=5, shuffle=True, random_state=1121218)
# scores = cross_validate(
#     gr_reg, X, y, cv=kf, scoring="neg_mean_squared_log_error", n_jobs=-1
# )

In [3]:
# rmsle = np.sqrt(-scores["test_score"].mean())
# print(f"Base RMSLE: {rmsle:.5f}")

In [9]:
def Optuna_Performance_Visuvalization(study,error_bar=True):
  from optuna.visualization import plot_optimization_history

  plotly_config = {"staticPlot": True}

  fig = plot_optimization_history(study)
  fig.show(config=plotly_config)

  from optuna.visualization import plot_param_importances

  fig = plot_param_importances(study)
  fig.show(config=plotly_config)

In [5]:
 #study.best_trial.params

In [6]:
# gr_reg(**study.best_params)
# GradientBoostingRegressor(**study.best_params))

In [24]:
def Optuna_Tuning(model,n_trials,X,y,scoring):
    def objective(trial, X, y, cv, scoring):
      params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 5000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "subsample": trial.suggest_float("subsample", 0.5, 0.9, step=0.1),
        "max_features": trial.suggest_categorical(
            "max_features", ["auto", "sqrt", "log2"]
        ),
        "random_state": 1121218,
        "n_iter_no_change": 50,  # early stopping
        "validation_fraction": 0.05,
      }
      # Perform CV
      gr_reg = model(**params)
      scores = cross_validate(gr_reg, X, y, cv=cv, scoring=scoring, n_jobs=-1)
      # Compute RMSLE
      rmsle = np.sqrt(-scores["test_score"].mean())
      print('rmsle',rmsle)
      return rmsle
  
    # kf = KFold(n_splits=5, shuffle=True, random_state=1121218)
    # scores = cross_validate(
    #    gr_reg, X, y, cv=kf, scoring="neg_mean_squared_log_error", n_jobs=-1
    #  )
    # Create study that minimizes
    study = optuna.create_study(direction="minimize")
    # Wrap the objective inside a lambda with the relevant arguments
    kf = KFold(n_splits=5, shuffle=True, random_state=1121218)
    # Pass additional arguments inside another function
    func = lambda trial: objective(trial, X, y, cv=kf, scoring=scoring)
    # Start optimizing with 100 trials
    study.optimize(func, n_trials=n_trials)

    print("Best params:")
    for key, value in study.best_params.items():
        print(f"\t{key}: {value}")

    Optuna_Performance_Visuvalization(study)

    return study.best_trial.params

In [27]:
#Optuna_Tuning(GradientBoostingRegressor,10,"neg_mean_squared_log_error",X,y)

Optuna_Tuning(GradientBoostingRegressor,25,X,y,"neg_mean_squared_log_error")

[I 2023-03-22 15:22:03,394] A new study created in memory with name: no-name-24d6e1c2-59a2-4c49-8a67-1d71f4edb1ef
[I 2023-03-22 15:22:04,356] Trial 0 finished with value: 0.07980721242372978 and parameters: {'n_estimators': 3200, 'learning_rate': 0.0510505284718396, 'max_depth': 8, 'subsample': 0.7, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.07980721242372978.


rmsle 0.07980721242372978


[I 2023-03-22 15:22:05,575] Trial 1 finished with value: 0.08500545234358735 and parameters: {'n_estimators': 1200, 'learning_rate': 0.05921857895293527, 'max_depth': 7, 'subsample': 0.7, 'max_features': 'auto'}. Best is trial 0 with value: 0.07980721242372978.


rmsle 0.08500545234358735


[I 2023-03-22 15:22:06,651] Trial 2 finished with value: 0.07849305336893408 and parameters: {'n_estimators': 4300, 'learning_rate': 0.013291428907080039, 'max_depth': 8, 'subsample': 0.6, 'max_features': 'log2'}. Best is trial 2 with value: 0.07849305336893408.


...

[I 2023-03-22 15:23:07,341] Trial 23 finished with value: 0.0748126359237529 and parameters: {'n_estimators': 600, 'learning_rate': 0.034547857625326084, 'max_depth': 4, 'subsample': 0.8, 'max_features': 'log2'}. Best is trial 17 with value: 0.0735098953304998.


rmsle 0.0748126359237529


[I 2023-03-22 15:23:08,322] Trial 24 finished with value: 0.07224911231355037 and parameters: {'n_estimators': 500, 'learning_rate': 0.027567875479567763, 'max_depth': 3, 'subsample': 0.9, 'max_features': 'log2'}. Best is trial 24 with value: 0.07224911231355037.


rmsle 0.07224911231355037
Best params:
	n_estimators: 500
	learning_rate: 0.027567875479567763
	max_depth: 3
	subsample: 0.9
	max_features: log2


{'n_estimators': 500,
 'learning_rate': 0.027567875479567763,
 'max_depth': 3,
 'subsample': 0.9,
 'max_features': 'log2'}

In [19]:
n_trials=10
scoring='neg_mean_squared_log_error'
model=GradientBoostingRegressor
def Optuna_Tuning(model,n_trials,X,y,scoring):
  def objective(trial, X, y, cv, scoring):
      params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 5000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "subsample": trial.suggest_float("subsample", 0.5, 0.9, step=0.1),
        "max_features": trial.suggest_categorical(
            "max_features", ["auto", "sqrt", "log2"]
        ),
        "random_state": 1121218,
        "n_iter_no_change": 50,  # early stopping
        "validation_fraction": 0.05,
      }
      # Perform CV
      gr_reg = model(**params)
      scores = cross_validate(gr_reg, X, y, cv=cv, scoring=scoring, n_jobs=-1)
      # Compute RMSLE
      score = np.sqrt(-scores["test_score"].mean())
      print('score',score)
      return score
  
    # kf = KFold(n_splits=5, shuffle=True, random_state=1121218)
    # scores = cross_validate(
    #    gr_reg, X, y, cv=kf, scoring="neg_mean_squared_log_error", n_jobs=-1
    #  )
    # Create study that minimizes
  study = optuna.create_study(direction="minimize")

    # Wrap the objective inside a lambda with the relevant arguments
  kf = KFold(n_splits=5, shuffle=True, random_state=1121218)
    # Pass additional arguments inside another function
  func = lambda trial: objective(trial, X, y, cv=kf, scoring=scoring)

    # Start optimizing with 100 trials
  study.optimize(func, n_trials=n_trials)

  print("Best params:")
  for key, value in study.best_params.items():
        print(f"\t{key}: {value}")

  Optuna_Performance_Visuvalization()

  return study.best_trial.params

In [20]:
print(f"Base RMSLE     : {rmsle:.5f}")
print(f"Optimized RMSLE: {study.best_value:.5f}")

NameError: ignored

In [21]:
%%time

study.optimize(func, n_trials=200)

NameError: ignored

In [22]:
print("Best params:")
for key, value in study.best_params.items():
    print(f"\t{key}: {value}")

Best params:


NameError: ignored

In [14]:
print(f"Base RMSLE     : {rmsle:.5f}")
print(f"Optimized RMSLE: {study.best_value:.5f}")

NameError: ignored

In [15]:
#Using visuals for more insights and smarter tuning¶
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: ignored

In [16]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

NameError: ignored